In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/LowRankVortex.jl/")

  Activating environment at `/media/mat/HDD/LowRankVortex.jl/Project.toml`


In [5]:
using Revise
using LowRankVortex
using LinearAlgebra
using Statistics
using PotentialFlow
import PotentialFlow.Plates: Plate, Points, Blobs
import PotentialFlow.Motions: reset_velocity!
import PotentialFlow.Elements
import PotentialFlow.Properties: @property
using Interpolations
using Distributions
using Test

In [17]:
@testset "Test symmetric version of pressure for point vortices" begin
    atol = 1000*eps()
    Nv = 10
    zv = randn(Nv) + im *(0.05 .+ rand(Nv))
    Sv = randn(Nv)

    points₊ = Vortex.Point.(zv, Sv)
    points₋ = Vortex.Point.(conj(zv), -Sv)

    sys = deepcopy(vcat(points₊, points₋))

    xsensors = collect(-2.0:0.1:2.0)
    sensors = complex(xsensors)
    Ny = size(xsensors, 1)

    press_sym = symmetric_pressure(xsensors, points₊, 0.0)

    press = pressure_AD(sensors, deepcopy(sys), 0.0)
    @test isapprox(press_sym, press, atol = atol)
end

Test Summary:                                         | Pass  Total
Test symmetric version of pressure for point vortices |    1      1


Test.DefaultTestSet("Test symmetric version of pressure for point vortices", Any[], 1, false, false)

In [32]:
@testset "Test symmetric version of pressure for point vortices with freestream" begin
    atol = 1000*eps()
    
    U = randn(ComplexF64)
    freestream = Freestream(U)
    Nv = 10
    zv = randn(Nv) + im *(0.05 .+ rand(Nv))
    Sv = randn(Nv)

    points₊ = Vortex.Point.(zv, Sv)
    points₋ = Vortex.Point.(conj(zv), -Sv)

    sys = deepcopy(vcat(points₊, points₋))

    xsensors = collect(-2.0:0.1:2.0)
    sensors = complex(xsensors)
    Ny = size(xsensors, 1)

    press_sym = symmetric_pressure(xsensors, points₊, freestream, 0.0)

    press_AD = pressure_AD(sensors, deepcopy(sys), freestream, 0.0)
    press = pressure(sensors, deepcopy(sys), freestream, 0.0)

#     @test isapprox(press_AD, press, atol = atol)
    @test isapprox(press_sym, press, atol = atol)

end

Test symmetric version of pressure for point vortices with freestream: Test Failed at In[32]:25
  Expression: isapprox(press_sym, press, atol = atol)
   Evaluated: isapprox([1.1018251405197745, 1.1088858272214372, 1.1167090348145141, 1.1286698557698782, 1.1514498994517215, 1.1969167273132344, 1.2848607851308207, 1.4481244780852596, 1.7420324816666772, 2.2217929479408216  …  0.7853036195633603, 0.7379862749158392, 0.8146070648803005, 0.9136640580047379, 0.8086577208010017, 0.6908790694451976, 0.6139500613514056, 0.5682801512602718, 0.5431653115327715, 0.5311960687166943], [-0.6273506913126075, -0.620223873430695, -0.6151282857342583, -0.6134507926190377, -0.6170523116662443, -0.6291672971822817, -0.6578241218906056, -0.7268223219187955, -0.9072574117754494, -1.3651568907420386  …  -0.7820309871598049, -0.7817170005351439, -0.816073653933382, -0.7965019801255261, -0.7305852789151673, -0.701018772872191, -0.6915002290330816, -0.6899619731655476, -0.6917878563240271, -0.6949821885850286]; 

LoadError: [91mSome tests did not pass: 0 passed, 1 failed, 0 errored, 0 broken.[39m

In [19]:
@testset "Test symmetric version of pressure for regularized point vortices" begin
    atol = 1000*eps()
    Nv = 10
    zv = randn(Nv) + im *rand(Nv)
    Sv = randn(Nv)
    δ = 0.5

    blobs₊ = Vortex.Blob.(zv, Sv, δ*ones(Nv))
    blobs₋ = Vortex.Blob.(conj(zv), -Sv, δ*ones(Nv))

    sys = deepcopy(vcat(blobs₊, blobs₋))


    xsensors = collect(-2.0:0.1:2.0)
    sensors = complex(xsensors)
    Ny = size(xsensors, 1)

    press_sym = symmetric_pressure(xsensors, blobs₊, 0.0)

    press = pressure_AD(sensors, deepcopy(sys), 0.0)
    @test isapprox(press_sym, press, atol = atol)
end

Test Summary:                                                     | Pass  Total
Test symmetric version of pressure for regularized point vortices |    1      1


Test.DefaultTestSet("Test symmetric version of pressure for regularized point vortices", Any[], 1, false, false)